In [1]:
from nariflow import start_initializer

start_initializer().initializer('tape')

In [2]:
from nariflow import Variable
from nariflow import optimizer
from nariflow import GradientTape
#from nariflow import calc_gradient
from nariflow import layer
from nariflow.models import Model
from nariflow import functions as f
from nariflow.core import elementary_function as ef
from nariflow.core import shape_function as sf
from nariflow.utils import DataSet
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from tensorflow.keras.layers.experimental import preprocessing

In [3]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

dataset_path = tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

In [5]:
# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop un-used columns.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

In [6]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

7383 train examples
1846 validation examples
2308 test examples


In [7]:
train_processing = pd.concat([pd.get_dummies(train.loc[:, train.apply(lambda x : x.dtype == 'object')]),
           train.loc[:,train.apply(lambda x : x.dtype != 'object')].apply(lambda x : (x - min(x)) / (max(x) - min(x)) )], 
          axis = 1)

In [8]:
X_train = train_processing.drop(['target'], axis = 1).to_numpy()

In [9]:
test_processing = pd.concat([pd.get_dummies(test.loc[:, test.apply(lambda x : x.dtype == 'object')]),
           test.loc[:,test.apply(lambda x : x.dtype != 'object')].apply(lambda x : (x - min(x)) / (max(x) - min(x)) )], 
          axis = 1)

test_processing = pd.concat([pd.DataFrame(columns = train_processing.columns),
           test_processing])[train_processing.columns].fillna(0)

In [10]:
y_test = np.array(pd.get_dummies(test_processing.loc[:,'target']))

In [11]:
X_test= test_processing.drop(['target'], axis = 1).to_numpy()

In [12]:
y_train = np.array(pd.get_dummies(train_processing.loc[:,'target']))

In [13]:
lr = 0.2
max_iter = 10000
hidden_size = 200
out_size = 2

In [14]:
class Models(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        self.l1 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l2 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l3 = layer.Linear(out_size, initializer_func='he_uniform')
        
    def forward(self, x):
        y = self.l1(x)
        y = f.activation.relu(y)
        y = self.l2(y)
        y = f.activation.relu(y)
        y = self.l3(y)
        y = f.activation.softmax(y)
        return y

In [15]:
lr = 0.001

In [16]:
model = Models(hidden_size, out_size)

optimizers = optimizer.Adam(lr)

optimizers.setup(model)

In [17]:
batch_size = 50

In [18]:
X_train = Variable(np.array(X_train))

y_train = Variable(y_train)

dataset = DataSet(X_train, y_train)

dataset.batch_setup(batch_size)

In [19]:
def train_step(X_train, y_train):
    with GradientTape() as tape:
        
        y_pred = model(X_train)
        loss = f.loss.categorical_crossentropy(y_train, y_pred)
        
    tape.CalcGradient()
    optimizers.update()
    
    if j % 5 == 0:
        print(loss.data)

In [20]:
start_time = time.time()

i = 0

while True:
    for j in range(np.ceil((len(dataset) / batch_size)).astype('int')):

        train_set, val_set = next(dataset)

        train_step(train_set, val_set)

    print('total_time : ', time.time() - start_time)

    dataset.reset()
    
    i += 1
    
    print(i)
    
    if i > 2:
        break

0.6931407228640074
0.6741362090424373
0.6272608113703272
0.6048838802298484
0.5580642166656314
0.5475145793575171
0.6273634052601446
0.6069214026752011
0.5712169291277811
0.5981289861829943
0.5476197220981411
0.5421274577317076
0.5987641682999968
0.5799741633662309
0.49922529881181066
0.60591609933516
0.6187848188513365
0.6085326457043315
0.5585452778533283
0.6318999678067682
0.6038626291823477
0.6122277782852593
0.5790797235995274
0.6091184825342222
0.5018586994665484
0.5317711946542125
0.5268654167353759
0.6085641482254808
0.6133390625239477
0.5572131517983423
total_time :  0.33904194831848145
1
0.6214051107974332
0.5399282172150237
0.6129749685117989
0.5845883284979599
0.5323828580453936
0.5212409276851995
0.5947679284889317
0.5824759164788078
0.5621924842762374
0.5621053927626063
0.5118860209193236
0.5336006183090586
0.5701090037207992
0.5619998082488208
0.497694653462761
0.5748429128753142
0.6101357784207604
0.5938733820326373
0.5404350955162883
0.6300680992410372
0.59020124147283

In [21]:
X_test = Variable(np.array(X_test))

In [22]:
y_pred = model(X_test)

In [23]:
f1_score(np.argmax(y_pred.data, axis = -1), 
               np.argmax(y_test.data, axis = -1))

0.8489751887810141

In [24]:
accuracy_score(np.argmax(y_pred.data, axis = -1), 
               np.argmax(y_test.data, axis = -1))

0.75736568457539

In [25]:
confusion_matrix(np.argmax(y_pred.data, axis = -1),
               np.argmax(y_test.data, axis = -1))

array([[ 174,  120],
       [ 440, 1574]], dtype=int64)

### 텐서플로우 공식 도큐의 Accuracy는 74%이다.
### 결론 : 다중분류도 문제없이 작동한다.